In [2]:
import os
os.chdir("../")

In [3]:
from os import read
from classifier_app.utils import read_yaml, create_directories, save_json
from pathlib import Path
from classifier_app.constants import *


config = read_yaml(Path("configs/config.yaml"))
evaluation_config = config.evaluation



In [4]:
import tensorflow as tf



evaluation_config
model = tf.keras.models.load_model(evaluation_config.trained_model_path)


In [5]:
### entity

from dataclasses import dataclass


@dataclass
class EvaluateConfig:
    trained_model_path: Path
    root_dir: Path
    test_data_path: Path
    params_target_size: list
    params_batch_size: int
    params_interpolation : str

   



In [6]:
## configuration manager

from classifier_app.constants import *

class ConfigurationManager:

    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    

    def get_evaluation_config(self):
        evaluation_config = self.config.evaluation
        
        evaluation_config = EvaluateConfig(
                                trained_model_path= evaluation_config.trained_model_path,
                                root_dir= evaluation_config.root_dir,
                                test_data_path= Path(os.path.join(self.config.data_ingestion.split_data, self.config.data_ingestion.test_dir_name)),
                                params_target_size= self.params.IMAGE_SIZE[:-1],
                                params_batch_size= self.params.BATCH_SIZE,
                                params_interpolation= self.params.INTERPOLATION
        )

        return evaluation_config


In [7]:
class Evaluate_Model:

    def __init__(self, config: EvaluateConfig):
        self.config = config
        print(self.config)
    
    def _load_model(self):
        self.model  = tf.keras.models.load_model(self.config.trained_model_path)

    def _test_generator(self):

        test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                            rescale= 1/255.0
        )

        test_generator = test_datagen.flow_from_directory(
                                directory= self.config.test_data_path,
                                target_size= self.config.params_target_size,
                                interpolation= self.config.params_interpolation,
                                batch_size= self.config.params_batch_size
        )
        return test_generator

    def evaluateModel(self):
        self._load_model()
        test_generator = self._test_generator()
        self.score  = self.model.evaluate(test_generator)
        return self.score

    def save_score(self):
        score_path = Path("scores.json")
        save_json(path_to_save=score_path, data= self.score)
    
            
        

In [8]:
## pipeline


try:
    config = ConfigurationManager()
    evaluate_config = config.get_evaluation_config()
    evaluation = Evaluate_Model(config=evaluate_config)
    score = evaluation.evaluateModel()
    

except Exception as e:
    raise e

EvaluateConfig(trained_model_path='artifacts/training/model.h5', root_dir='artifacts/evaluation', test_data_path=WindowsPath('artifacts/data_ingestion/splitted_data/test'), params_target_size=[224, 224], params_batch_size=8, params_interpolation='bilinear')
Found 7500 images belonging to 2 classes.
 21/938 [..............................] - ETA: 26:40 - loss: 2.1077 - accuracy: 0.8512

In [ ]:
print(score)
# evaluation.save_score()